In [ ]:
import os
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import pickle
import numpy as np
import wandb
import pandas as pd 
from pprint import pprint
from copy import deepcopy
from dataclasses import dataclass, field
import matplotlib
import matplotlib.pyplot as plt
import re
from pprint import pprint

In [ ]:

# save_dir = '/dev_data/zjh/LLM_CL/experiments/recovered_models/step19_minus_initialization'

# alpha = 0.75
step_id = 19

backbone = 'EleutherAI/pythia-160m-deduped'
path_pretrained = f'/dev_data/zjh/physics-of-forgetting-in-llm/physics_of_forgetting/model/gpt-neox/v_0720/multi5_permute_fullname/full_fine_tuning/lr_5e-06_wd_0.01/final_model'  # model pre-trained on 100k biography data, fine-tuned on task0
# path_step = f'/dev_data/zjh/LLM_CL/experiments/Outdomain-Finetuned-tmp_SEQ_epoch_1_bs_240_epoch_25_lr5e-5/2024-08-07-22-07-33/checkpoint_llm_task_0_epoch_0_step_{step_id}'  # model fine-tuned on task1 (epoch_1 lr5e-5)
for e in [20, 22, 24]:
    path_step = f'/dev_data/zjh/LLM_CL/experiments/Outdomain-Finetuned-tmp_SEQ_bs_240_epoch_25_lr5e-5/2024-08-07-12-52-52/checkpoint_llm_task_0_epoch_{e}'
    save_dir = f'/dev_data/zjh/LLM_CL/experiments/recovered_models/epoch{e}_minus_initialization'

    tokenizer = AutoTokenizer.from_pretrained(backbone)

    config = AutoConfig.from_pretrained(path_pretrained)
    model_pretrained = AutoModelForCausalLM.from_pretrained(path_pretrained, config=config)  # get the pre-trained model

    config = AutoConfig.from_pretrained(path_step)
    model_step = AutoModelForCausalLM.from_pretrained(path_step, config=config)  # ge the model that is fine-tuned on task1 for several steps (currently 19 steps)

    for alpha in [0.16, 0.18, 0.2, 0.22, 0.24, 0.4, 0.6, 0.8, 1.0]:
        for epoch_id in range(25):

            path_epoch = f'/dev_data/zjh/LLM_CL/experiments/Outdomain-Finetuned-tmp_SEQ_bs_240_epoch_25_lr5e-5/2024-08-07-12-52-52/checkpoint_llm_task_0_epoch_{epoch_id}'  # model fine-tuned on task1 (lr5e-5)

            config = AutoConfig.from_pretrained(path_epoch)
            model_epoch = AutoModelForCausalLM.from_pretrained(path_epoch, config=config)

            save_ckpt_name = f'model_epoch{epoch_id}_add_epoch{e}_minus_pretrained_alpha{alpha}'
            for (n1,p1), (n2,p2), (n3,p3) in zip(model_pretrained.named_parameters(), 
                                                model_step.named_parameters(), 
                                                model_epoch.named_parameters()):
                assert n1==n2 and n2==n3
                p3.data = p3.data-alpha*(p2.data-p1.data)
            save_path = os.path.join(save_dir, save_ckpt_name)
            if os.path.exists(save_path):
                raise ValueError(f"{save_ckpt_name} already exists.")
            model_epoch.save_pretrained(save_path)

In [ ]:
for epoch_of_task_vector in [20, 22, 24]:
    task_vector_start_dir = f'/dev_data/zjh/physics-of-forgetting-in-llm/physics_of_forgetting/model/gpt-neox/v_0720/multi5_permute_fullname/full_fine_tuning/lr_5e-06_wd_0.01/final_model'
    config = AutoConfig.from_pretrained(task_vector_start_dir)
    task_vector_start_model = AutoModelForCausalLM.from_pretrained(task_vector_start_dir, config=config)

    task_vector_end_dir = f'/dev_data/zjh/LLM_CL/experiments/Outdomain-Finetuned-tmp_SEQ_bs_240_epoch_25_lr5e-5/2024-08-07-12-52-52/checkpoint_llm_task_0_epoch_{epoch_of_task_vector}'
    config = AutoConfig.from_pretrained(task_vector_end_dir)
    task_vector_end_model = AutoModelForCausalLM.from_pretrained(task_vector_end_dir, config=config)
    for epoch_ckpt in range(0, 25, 2):
        for alpha in [0.16, 0.18, 0.2, 0.22, 0.24, 0.4, 0.6, 0.8, 1.0]:
            model_to_be_modified_dir = f'/dev_data/zjh/LLM_CL/experiments/Outdomain-Finetuned-tmp_SEQ_bs_240_epoch_25_lr5e-5/2024-08-07-12-52-52/checkpoint_llm_task_0_epoch_{epoch_ckpt}'
            config = AutoConfig.from_pretrained(model_to_be_modified_dir)
            model_to_be_modified = AutoModelForCausalLM.from_pretrained(model_to_be_modified_dir, config=config)

            for (n_start, p_start), (n_end, p_end), (n, p) in zip(task_vector_start_model.named_parameters(), task_vector_end_model.named_parameters(), model_to_be_modified.named_parameters()):
                assert n_start==n_end and n_end==n
                p.data = p.data - alpha * (p_end.data - p_start.data)
            save_dir = f'/dev_data/zjh/LLM_CL/experiments/recovered_models/epoch{epoch_of_task_vector}_minus_initialization'
            save_ckpt_name = f'model_epoch{epoch_ckpt}_add_epoch{epoch_of_task_vector}_minus_pretrained_alpha{alpha}'
            save_path = os.path.join(save_dir, save_ckpt_name)
            if os.path.exists(save_path):
                raise ValueError(f"{save_ckpt_name} already exists.")
            model_to_be_modified.save_pretrained(save_path)

for epoch_of_task_vector in [20, 22, 24]:
    # task_vector_start_dir = f'/dev_data/zjh/physics-of-forgetting-in-llm/physics_of_forgetting/model/gpt-neox/v_0720/multi5_permute_fullname/full_fine_tuning/lr_5e-06_wd_0.01/final_model'
    task_vector_start_dir = f'/dev_data/zjh/LLM_CL/experiments/Outdomain-Finetuned-tmp_SEQ_epoch_1_bs_240_epoch_25_lr5e-5/2024-08-07-22-07-33/checkpoint_llm_task_0_epoch_0_step_19'
    config = AutoConfig.from_pretrained(task_vector_start_dir)
    task_vector_start_model = AutoModelForCausalLM.from_pretrained(task_vector_start_dir, config=config)

    task_vector_end_dir = f'/dev_data/zjh/LLM_CL/experiments/Outdomain-Finetuned-tmp_SEQ_bs_240_epoch_25_lr5e-5/2024-08-07-12-52-52/checkpoint_llm_task_0_epoch_{epoch_of_task_vector}'
    config = AutoConfig.from_pretrained(task_vector_end_dir)
    task_vector_end_model = AutoModelForCausalLM.from_pretrained(task_vector_end_dir, config=config)
    for epoch_ckpt in range(0, 25, 2):
        for alpha in [0.16, 0.18, 0.2, 0.22, 0.24, 0.4, 0.6, 0.8, 1.0]:
            model_to_be_modified_dir = f'/dev_data/zjh/LLM_CL/experiments/Outdomain-Finetuned-tmp_SEQ_bs_240_epoch_25_lr5e-5/2024-08-07-12-52-52/checkpoint_llm_task_0_epoch_{epoch_ckpt}'
            config = AutoConfig.from_pretrained(model_to_be_modified_dir)
            model_to_be_modified = AutoModelForCausalLM.from_pretrained(model_to_be_modified_dir, config=config)

            for (n_start, p_start), (n_end, p_end), (n, p) in zip(task_vector_start_model.named_parameters(), task_vector_end_model.named_parameters(), model_to_be_modified.named_parameters()):
                assert n_start==n_end and n_end==n
                p.data = p.data - alpha * (p_end.data - p_start.data)
            save_dir = f'/dev_data/zjh/LLM_CL/experiments/recovered_models/epoch{epoch_of_task_vector}_minus_step19'
            save_ckpt_name = f'model_epoch{epoch_ckpt}_add_epoch{epoch_of_task_vector}_minus_step19_alpha{alpha}'
            save_path = os.path.join(save_dir, save_ckpt_name)
            if os.path.exists(save_path):
                raise ValueError(f"{save_ckpt_name} already exists.")
            model_to_be_modified.save_pretrained(save_path)

In [ ]:
# collect Recovered-Evaluate-IND-Tradeoff
# /dev_data/zjh/LLM_CL/experiments/Recovered-Evaluate-IND-Tradeoff-epoch24-add-epoch18-alpha1.0-SEQ_epoch_1
all_column_list = []
all_acc_list = []
for epoch_of_task_vector in [12, 14, 16, 18]:
    for alpha in [0.2, 1.0]:
        # print(f'epoch_of_task_vector: {epoch_of_task_vector}, alpha: {alpha}')
        all_column_list.append(f'epoch_of_task_vector_{epoch_of_task_vector}_alpha_{alpha}')
        task_0_acc_list = []
        for epoch_ckpt in range(0, 26, 2):
            path = f'/dev_data/zjh/LLM_CL/experiments/Recovered-Evaluate-IND-Tradeoff-epoch{epoch_ckpt}-add-epoch{epoch_of_task_vector}-alpha{alpha}-SEQ_epoch_1'
            all_dir = os.listdir(path)
            path = os.path.join(path, all_dir[0])
            matrix_path = os.path.join(path, 'test_cur_task_0_save_result.npy')
            train_log_path = os.path.join(path, 'train.log')
            matrix = np.load(matrix_path, allow_pickle=True)  # get [None] 😢
            with open(train_log_path, 'r') as f:
                train_log = f.readlines()
                for line in train_log:
                    # use regular expression to search for pattern: Test Result = {'Test_Acc_Task_0': 99.626, 'Test_Acc_Task_Seen': 99.626, 'Test_Acc_Task_All': 99.626}
                    match = re.search(r"Test Result = \{'Test_Acc_Task_0': (\d+\.\d+), 'Test_Acc_Task_Seen': (\d+\.\d+), 'Test_Acc_Task_All': (\d+\.\d+)\}", line)
                    if match:
                        task_0_acc = match.group(1)
                        task_0_acc_list.append(float(task_0_acc))
                        break
                else:
                    raise ValueError(f"Test Result not found in {train_log_path}")
        all_acc_list.append(task_0_acc_list)
print(all_acc_list)
all_acc_arr = np.array(all_acc_list).T
result_df = pd.DataFrame(all_acc_arr, columns=all_column_list)
print(result_df)
result_df.to_csv('/dev_data/zjh/LLM_CL/visualization-tradeoff/temp.csv')


In [ ]:
# collect Recovered-Evaluate-Tradeoff-Sample
# /dev_data/zjh/LLM_CL/experiments/Recovered-Evaluate-Tradeoff-Sample-epoch24-add-epoch18-alpha1.0-tmp_Evaluate_bs_1440_sample_2000
task0_column_list = []
task0_acc_list = []
task1_column_list = []
task1_acc_list = []
for epoch_of_task_vector in [12, 14, 16, 18]:
    for alpha in [0.16, 0.18, 0.2, 0.22, 0.24, 0.4, 0.6, 0.8, 1.0]:
        task0_column_list.append(f'epoch_of_task_vector_{epoch_of_task_vector}_alpha_{alpha}')
        task1_column_list.append(f'epoch_of_task_vector_{epoch_of_task_vector}_alpha_{alpha}')
        _task0_acc_list = []
        _task1_acc_list = []
        for epoch_ckpt in range(0, 26, 2):
            path = f'/dev_data/zjh/LLM_CL/experiments/Recovered-Evaluate-Tradeoff-Sample-epoch{epoch_ckpt}-add-epoch{epoch_of_task_vector}-alpha{alpha}-tmp_Evaluate_bs_1440_sample_2000'
            all_dir = os.listdir(path) 
            path = os.path.join(path, all_dir[0])
            train_log_path = os.path.join(path, 'train.log')
            with open(train_log_path, 'r') as f:
                train_log = f.readlines()
                for line in train_log:
                    # find Test Result = {'Test_Acc_Task_0': 99.775, 'Test_Acc_Task_1': 0.875, 'Test_Acc_Task_2': 0.0, 'Test_Acc_Task_3': 16.667, 'Test_Acc_Task_4': 0.0, 'Test_Acc_Task_5': 0.0, 'Test_Acc_Task_Seen': 19.553, 'Test_Acc_Task_All': 19.553}
                    match = re.search(r"Test Result = \{'Test_Acc_Task_0': (\d+\.\d+), 'Test_Acc_Task_1': (\d+\.\d+), 'Test_Acc_Task_2': (\d+\.\d+), 'Test_Acc_Task_3': (\d+\.\d+), 'Test_Acc_Task_4': (\d+\.\d+), 'Test_Acc_Task_5': (\d+\.\d+), 'Test_Acc_Task_Seen': (\d+\.\d+), 'Test_Acc_Task_All': (\d+\.\d+)\}", line)
                    if match:
                        task0_acc, task1_acc = match.group(1), match.group(2)
                        _task0_acc_list.append(float(task0_acc))
                        _task1_acc_list.append(float(task1_acc))
                        break
                else:
                    raise ValueError(f"Test Result not found in {train_log_path}")
        task0_acc_list.append(_task0_acc_list)
        task1_acc_list.append(_task1_acc_list)
print(task0_acc_list)
print(task1_acc_list)
task0_acc_arr = np.array(task0_acc_list).T
task1_acc_arr = np.array(task1_acc_list).T
task0_result_df = pd.DataFrame(task0_acc_arr, columns=task0_column_list)
task1_result_df = pd.DataFrame(task1_acc_arr, columns=task1_column_list)
print(task0_result_df)
print(task1_result_df)
task0_result_df.to_csv('/dev_data/zjh/LLM_CL/visualization-tradeoff/task0_temp.csv')
task1_result_df.to_csv('/dev_data/zjh/LLM_CL/visualization-tradeoff/task1_temp.csv')